In [1]:
#!pip install web3

In [2]:
#!pip install multicall

**Gearbox Protocol**

Website:        https://gearbox.fi/ <br>
dApp:           https://app.gearbox.fi/ <br>
Docs:           https://docs.gearbox.finance/ <br>
Developer Docs: https://dev.gearbox.fi/ <br>

**Gearbox Credit Accout Report** <br>
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from gearbox_ca import (pull_abi_etherscan, parse_abi, chunks, get_data_multicall, get_logs, get_token_balance, getCreditAccountData) # see gearbox_ca.py
import os
from datetime import datetime
from pprint import pformat
import logging
import pandas as pd
from web3 import Web3
from web3.datastructures import AttributeDict
from web3.exceptions import (ContractLogicError, ABIFunctionNotFound)

######### Enable logging for debug
logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level='INFO',
        datefmt='%Y-%m-%d %H:%M:%S',
        )


Etherscan_APIKEY   = None #optional, but recommended
GearboxAddressProvider = '0xcF64698AFF7E5f27A11dff868AF228653ba53be0'

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 30}))
logging.info (f'Ethereum connected: {w3_eth.isConnected()}')


2023-07-09 21:48:17 <module> INFO Ethereum connected: True


In [5]:
logging.info('Start')
AddressProvider = Web3.toChecksumAddress(GearboxAddressProvider)
AddressProvider_abi = pull_abi_etherscan(AddressProvider)

AccountFactory = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getAccountFactory().call()
logging.info(f'AccountFactory: {AccountFactory}')
AccountFactory_abi = pull_abi_etherscan(AccountFactory)
countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccounts().call()
countCreditAccountsInStock = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccountsInStock().call()
logging.info(f'countCreditAccounts: {countCreditAccounts}')
logging.info(f'countCreditAccountsInStock: {countCreditAccountsInStock}')
masterCreditAccount = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.masterCreditAccount().call()
logging.info(f'masterCreditAccount: {masterCreditAccount}')
CreditAccount_abi = pull_abi_etherscan(masterCreditAccount)

DataCompressor = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getDataCompressor().call()
logging.info(f'DataCompressor: {DataCompressor}')
DataCompressor_abi = pull_abi_etherscan(DataCompressor)

ContractsRegister = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getContractsRegister().call()
logging.info(f'ContractsRegister: {ContractsRegister}')
ContractsRegister_abi = pull_abi_etherscan(ContractsRegister)

CreditManagers = w3_eth.eth.contract(address=ContractsRegister, abi=ContractsRegister_abi).functions.getCreditManagers().call()
logging.info(f'CreditManagers: {CreditManagers}')
version = [None] * len(CreditManagers)

cm_dict = {AccountFactory: {'token': None, 'symbol': None, 'decimals': None, 'version': None,'creditFacade': None}}
allowedTokens = {}
CreditManager_abi = Token_abi = CreditFilter_abi = creditFacade_abi = priceOracle_abi = creditConfigurator_abi = ''

for i, CreditManager in enumerate(CreditManagers):
    logging.info(f'CreditManager {i+1} of {len(CreditManagers)}')
    
    if not CreditManager_abi:
        CreditManager_abi = pull_abi_etherscan(CreditManager)

    version[i] = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.version().call() 
    
    if i>0 and version[i]!=version[i-1]:
        CreditManager_abi = pull_abi_etherscan(CreditManager)
    
    logging.info(f'version {version[i]}')
    
    
    if version[i] == 1:
        Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlyingToken().call()  
        CreditFilter = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFilter().call()
        if not CreditFilter_abi:
            CreditFilter_abi = pull_abi_etherscan(CreditFilter) 
        allowedTokensCount = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokensCount().call()  
        priceOracle        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.priceOracle().call()
        wethAddress        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.wethAddress().call()
        CreditManager_v1_abi = CreditManager_abi
        creditFacade = None
    else:
        Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlying().call() 
        allowedTokensCount = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.collateralTokensCount().call()
        priceOracle        = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.priceOracle().call()
        wethAddress        = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.wethAddress().call()

        creditFacade = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFacade().call()
        if not creditFacade_abi:
            creditFacade_abi = pull_abi_etherscan(creditFacade)
            
        if not creditConfigurator_abi:
            creditConfigurator = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditConfigurator().call()
            creditConfigurator_abi = pull_abi_etherscan(creditConfigurator)
    
    if not Token_abi:
        Token_abi = pull_abi_etherscan(Token) 
    
    if not priceOracle_abi or version[i]!=version[i-1]:
        priceOracle_abi = pull_abi_etherscan(priceOracle)
    
    Token_symbol = w3_eth.eth.contract(address=Token, abi=Token_abi).functions.symbol().call()
    Token_decimals = w3_eth.eth.contract(address=Token, abi=Token_abi).functions.decimals().call()
    cm_dict[CreditManager] = {'version' : version[i],
                              'creditFacade': creditFacade,
                              'creditFacade_upgraded':[],
                              'token': Token,
                              'symbol': Token_symbol,
                              'decimals': Token_decimals,
                              'priceOracle' : priceOracle,
                              'allowedTokensCount': allowedTokensCount,
                              'allowedTokens':{},
                             }
    for token_id in range(allowedTokensCount):
        if version[i] == 1:
            allowed_token = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokens(token_id).call()
        else:
            allowed_token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.collateralTokens(token_id).call()
            allowed_token = allowed_token[0]
            
        allowed_token_symbol = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.symbol().call()
        allowed_token_decimals = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.decimals().call()
        
        
        
        if version[i] == 1:
            #try:
            #    allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, wethAddress).call()
            #    allowed_token_weth_priceOracle = allowed_token_weth_priceOracle*(10**allowed_token_decimals)/1e18
            #except ContractLogicError:
            #    allowed_token_weth_priceOracle = None
            #    logging.info(allowed_token_symbol+'-WETH getLastPrice error')
            #try:
            #    allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, Token).call()
            #    allowed_token_underlying_priceOracle = allowed_token_underlying_priceOracle*(10**allowed_token_decimals)/(10**Token_decimals)
            #except ContractLogicError:
            #    allowed_token_underlying_priceOracle = None
            #    logging.info(allowed_token_symbol+'-underlying getLastPrice error')
            USDC_Address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'    
            allowed_token_usd_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, USDC_Address).call()/1e18
        else:
            try:                
                #allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.convert(10**8, allowed_token, wethAddress).call()
                #allowed_token_weth_priceOracle = allowed_token_weth_priceOracle*10**(-8)
                #allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.convert(10**8, allowed_token, Token).call()
                #allowed_token_underlying_priceOracle = allowed_token_underlying_priceOracle*10**(-8)                

                allowed_token_usd_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getPrice(allowed_token).call()
                allowed_token_usd_priceOracle=allowed_token_usd_priceOracle*10**(-8) 
            except ContractLogicError:
                #allowed_token_usd_priceOracle = allowed_token_underlying_priceOracle= None
                allowed_token_usd_priceOracle = None
                logging.info(allowed_token_symbol+'- getPrice error')            
            
        Token_dict={'symbol'          : allowed_token_symbol, 
                    'decimals'        : allowed_token_decimals,
                    'Price_USD'       : allowed_token_usd_priceOracle,
                    #'Price_WETH'      : allowed_token_weth_priceOracle,
                    #'Price_'+cm_dict[CreditManager]['symbol']: allowed_token_underlying_priceOracle,
                    }

        if allowed_token in allowedTokens:
            allowedTokens[allowed_token].update(Token_dict)
        else:
            allowedTokens[allowed_token] = Token_dict
    
df_abi = parse_abi({'AddressProvider':AddressProvider_abi, 
                    'AccountFactory': AccountFactory_abi, 
                    'DataCompressor': DataCompressor_abi,
                    'CreditAccount': CreditAccount_abi,                     
                    'CreditManager': CreditManager_abi,
                    'creditFacade': creditFacade_abi,               
                    'CreditManager_v1': CreditManager_v1_abi,
                    'creditConfigurator':creditConfigurator_abi,
                   })

logging.info(pformat(cm_dict)) 
logging.info(pformat(allowedTokens))

2023-07-09 21:48:17 <module> INFO Start
2023-07-09 21:48:17 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2023-07-09 21:48:23 <module> INFO AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
2023-07-09 21:48:23 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2023-07-09 21:48:29 <module> INFO countCreditAccounts: 5001
2023-07-09 21:48:29 <module> INFO countCreditAccountsInStock: 4920
2023-07-09 21:48:30 <module> INFO masterCreditAccount: 0x373A292b93ff9017D28E64154ef83b99D5C4e270
2023-07-09 21:48:30 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2023-07-09 21:48:36 <module> INFO DataCompressor: 0x0a2CA503153Cd5CB2892a0928ac0F71F49a3c194
2023-07-09 21:48:36 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2023-07-09 21:48:42 <module> INFO ContractsRegister: 0xA50d4E7D8946a7c90652339CDBd262c375d54D99
2023-07-09 21:48:42 pull_abi_etherscan INFO e

2023-07-09 21:55:08 <module> INFO {'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd': {'Price_USD': 1.00167607,
                                                'decimals': 2,
                                                'symbol': 'GUSD'},
 '0x06325440D014e39736583c165C2963BA99fAf14E': {'Price_USD': 2010.26013319,
                                                'decimals': 18,
                                                'symbol': 'steCRV'},
 '0x0A1D4A25d0390899b90bCD22E1Ef155003EA76d7': {'Price_USD': 1.02450287,
                                                'decimals': 18,
                                                'symbol': 'stkcvxLUSD3CRV-f'},
 '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e': {'Price_USD': 7057.217207552814,
                                                'decimals': 18,
                                                'symbol': 'YFI'},
 '0x111111111117dC0aa78b770fA6A738034120C302': {'Price_USD': 0.3068821757035416,
                                               

In [6]:
df = pd.DataFrame()
df['id'] = range(countCreditAccounts)
for ids in list(chunks(list(df['id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    d_ca = get_data_multicall(w3_eth, df.loc[id_range], 'creditAccounts', df_abi[(df_abi['contract_type']=='AccountFactory')], AccountFactory)
    df.loc[id_range,'CA'] = df.loc[id_range].apply(lambda x: d_ca[x['id']], axis=1)

    d_cm = get_data_multicall(w3_eth, df.loc[id_range], 'creditManager', df_abi[(df_abi['contract_type']=='CreditAccount')])
    df.loc[id_range,'CM'] = df.loc[id_range].apply(lambda x: d_cm[x['id']], axis=1)
    
    d_amount = get_data_multicall(w3_eth, df.loc[id_range], 'borrowedAmount', df_abi[(df_abi['contract_type']=='CreditAccount')])
    df.loc[id_range,'borrowedAmount'] = df.loc[id_range].apply(lambda x: d_amount[x['id']] if d_amount[x['id']] > 1 else 0 , axis=1)
    
    d_since = get_data_multicall(w3_eth, df.loc[id_range], 'since', df_abi)
    df.loc[id_range,'Since'] = df.loc[id_range].apply(lambda x: d_since[x['id']], axis=1)

df['Since'] = df['Since'].astype(int)
df['Decimals'] = df.apply(lambda x: cm_dict[x['CM']]['decimals'], axis=1)
df['Symbol'] = df.apply(lambda x: cm_dict[x['CM']]['symbol'] , axis=1)
df['CF'] = df.apply(lambda x: cm_dict[x['CM']]['creditFacade'] , axis=1)

In [7]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI,None
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH,None
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI,None
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH,None
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,None
...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None


In [8]:
#df = df[(df['CA']=='0x24Ed6EB99C9AA7229336aaA05b4f3B07827404eA')]
#display(df)

#cm_dict['0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B']['creditFacade_upgraded'] = []

In [9]:
#newConfigurator, CreditFacadeUpgraded 
newConfigurator_topic = df_abi[(df_abi['name']=='NewConfigurator') & (df_abi['type']=='event')]['topic'].values[0]
CreditFacadeUpgraded_topic = df_abi[(df_abi['name']=='CreditFacadeUpgraded') & (df_abi['type']=='event')]['topic'].values[0]

#Open, Close, Repay, Liquidate
OpenCreditAccount_topic             =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
OpenCreditAccount_topic_v1          =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic                 =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic_v1              =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic            =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='RepayCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic        =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic_v1     =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateExpiredCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateExpiredCreditAccount')  & (df_abi['contract_type']=='creditFacade') &(df_abi['type']=='event')]['topic'].values[0]

logging.info(f'OpenCreditAccount_topics: {OpenCreditAccount_topic_v1}, {OpenCreditAccount_topic}' )
logging.info(f'CloseCreditAccount_topic: {CloseCreditAccount_topic_v1}, {CloseCreditAccount_topic}')
logging.info(f'RepayCreditAccount_topic: {RepayCreditAccount_topic_v1}')
logging.info(f'LiquidateCreditAccount_topic: {LiquidateCreditAccount_topic_v1}, {LiquidateCreditAccount_topic}')
logging.info(f'LiquidateExpiredCreditAccount_topic: {LiquidateExpiredCreditAccount_topic}')

logs = pd.DataFrame()
for CM in CreditManagers:
    
    if len(df.loc[df['CM']==CM]['Since'])>0:
        block_from = df.loc[df['CM']==CM]['Since'].min()
        
        CM_logs = get_logs(w3_eth, CM, df_abi,
                           [[OpenCreditAccount_topic_v1,
                             AddCollateral_topic_v1,
                             CloseCreditAccount_topic_v1,
                             RepayCreditAccount_topic_v1,
                             LiquidateCreditAccount_topic_v1,]
                             ],
                             block_from,
                             'latest')
        logs = logs.append(CM_logs, ignore_index = True)
        
        if cm_dict[CM]['creditFacade']:
            cm_dict[CM]['creditFacade_upgraded'] = []
            cc_upgraded = get_logs(w3_eth, CM, df_abi, [[newConfigurator_topic]], 0, 'latest')['args'].values
            cc_upgraded = [x['newConfigurator'] for x in cc_upgraded]
            
            for cc in cc_upgraded:
                cf_upgraded = get_logs(w3_eth, cc, df_abi, [[CreditFacadeUpgraded_topic]], 0, 'latest')['args'].values
                cf_upgraded = [x['newCreditFacade'] for x in cf_upgraded]
                
                for cf in cf_upgraded:
                    logging.info(f'CreditFacade: {cf}')
                    if cf not in cm_dict[CM]['creditFacade_upgraded']:
                        CF_logs = get_logs(w3_eth, cf, df_abi,
                                           [[OpenCreditAccount_topic,
                                             AddCollateral_topic, 
                                             CloseCreditAccount_topic,
                                             LiquidateCreditAccount_topic,
                                             LiquidateExpiredCreditAccount_topic,]
                                             ],
                                             block_from,
                                             'latest')
                        cm_dict[CM]['creditFacade_upgraded'] = cm_dict[CM]['creditFacade_upgraded']+[cf]
                        logs = logs.append(CF_logs, ignore_index = True)
                        
                        logging.info(f'CF_logs count: {len(CF_logs)}')                    

i=0
for row in df.loc[df['CM'].isin(CreditManagers)].itertuples():
    i+=1
    open_events       = logs[(logs['address'].isin([row.CM]+cm_dict[row.CM]['creditFacade_upgraded'])) & (logs['blockNumber']>=row.Since) & (logs['event']=='OpenCreditAccount')]['args'].values 
    collateral_events = logs[(logs['address'].isin([row.CM]+cm_dict[row.CM]['creditFacade_upgraded'])) & (logs['blockNumber']>=row.Since) & (logs['event']=='AddCollateral')]['args'].values 
    close_events      = logs[(logs['address'].isin([row.CM]+cm_dict[row.CM]['creditFacade_upgraded'])) & (logs['blockNumber']>=row.Since) & (logs['event']=='CloseCreditAccount')]['args'].values 
    repay_events      = logs[(logs['address'].isin([row.CM]+cm_dict[row.CM]['creditFacade_upgraded'])) & (logs['blockNumber']>=row.Since) & (logs['event']=='RepayCreditAccount')]['args'].values
    liquidate_event   = logs[(logs['address'].isin([row.CM]+cm_dict[row.CM]['creditFacade_upgraded'])) & (logs['blockNumber']>=row.Since) & (logs['event'].isin(['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']))]['args'].values

    CA_open_event = [x for x in open_events if x['creditAccount']== row.CA] # Open
    if len(CA_open_event) > 0:
        Borrower = CA_open_event[0]['onBehalfOf']
        df.loc[df['id']==row.id, 'Borrower'] = Borrower
        
        Collateral = sum([x['amount'] for x in CA_open_event if 'amount' in x])
        CA_collateral_event = [x for x in collateral_events if x['onBehalfOf']== Borrower] # Open
        if len(CA_collateral_event) > 0:
            Collateral = Collateral + sum(
                                            [(x['value']*(allowedTokens[x['token']]['Price_USD'])*(10**-allowedTokens[x['token']]['decimals']))
                                             /(allowedTokens[cm_dict[row.CM]['token']]['Price_USD']*(10**-row.Decimals))
                                            for x in CA_collateral_event 
                                            if allowedTokens[cm_dict[row.CM]['token']]['Price_USD']>0
                                            ]
                                         )
        df.loc[df['id']==row.id, 'Collateral'] = Collateral
                           
        CA_close_event = [x for x in close_events if x['to']== Borrower] # Close
        if len(CA_close_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_repay_event = [x for x in repay_events if x['to']== Borrower] # Repay
        if len(CA_repay_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_liquidate_event = [x for x in liquidate_event if ('owner' in x and x['owner']==Borrower) or ('borrower' in x and x['borrower']==Borrower)] # Liquidate
        if len(CA_liquidate_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]
            
    if i % 500 == 0:
        logging.info (i)

logging.info(f'{i} end')


2023-07-09 21:56:01 <module> INFO OpenCreditAccount_topics: 0x7b20ae77867a263a1074203a2da261ef0d096c99395c59c9d4a0104b9f334a27, 0xfa2baf5d3eb95569f312f22477b246f9d4c50276f1cb3ded8e1aeadcbc07a763
2023-07-09 21:56:01 <module> INFO CloseCreditAccount_topic: 0xca05b632388199c23de1352b2e96fd72a0ec71611683330b38060c004bbf0a76, 0x460ad03b1cf79b1d64d3aefa28475f110ab66e84649c52bb41ed796b9b391981
2023-07-09 21:56:01 <module> INFO RepayCreditAccount_topic: 0xe7c7987373a0cc4913d307f23ab8ef02e0333a2af445065e2ef7636cffc6daa7
2023-07-09 21:56:01 <module> INFO LiquidateCreditAccount_topic: 0x5e5da6c348e62989f9cfe029252433fc99009b7d28fa3c20d675520a10ff5896, 0x7dfecd8419723a9d3954585a30c2a270165d70aafa146c11c1e1b88ae1439064
2023-07-09 21:56:01 <module> INFO LiquidateExpiredCreditAccount_topic: 0xfada13ff2d5e1a1d9da37ad4a4130893b34e3c69c32b17f87ec31fd661b86707
2023-07-09 21:56:05 <module> INFO CreditFacade: 0xf6f4F24ae50206A07B8B32629AeB6cb1837d854F
2023-07-09 21:56:05 <module> INFO CF_logs count: 299
20

In [10]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,None,0.0
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,None,0.0
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,None,0.0
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,None,0.0
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13864162,18.0,WETH,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN


In [11]:
display(df[pd.notna(df['Borrower'])]) # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,None,0x896b94f4f27f12369698C302e2049cAe86936BbB,1.000000e+18
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,None,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,1.000000e+18
31,31,0xdAC87546b0ea6A448c7F3b69e079b0a616F4CC10,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,5000000000000000000,13868591,18.0,WETH,None,0x99BC02c239025E431D5741cC1DbA8CE77fc51CE3,2.500000e+18
33,33,0xB8c3FFFE2E13EFd96594EC6Ea9bBE0b2ABA6210e,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13869000,18.0,WETH,None,0x2bb8Ab3C2A9837de97A83c228A07E16928B4f07f,5.000000e+17
87,87,0xf74e4AEAA3B75c0662bF6024dcA385bF5fDBa8c0,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13884707,18.0,WETH,None,0x8CBC2D4c114D6f4411bBC991cE68Db994e45Eb4F,3.000000e+17
...,...,...,...,...,...,...,...,...,...,...
1363,1363,0x8137351530f9CD81C87EAe9457A6CAF9f910f000,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,468920089983699958680,17321436,18.0,WETH,0x67479449b2cf25AEE2fB6EF6f0aEc54591154F62,0x316BE293C8f2380769e7b7e7382679FE5a3b6600,9.999015e+19
1365,1365,0x2C0C914DC4ed0d850dDdb978ee04F4a3Ff445dc0,0x95357303f995e184A7998dA6C6eA35cC728A1900,300000000000,17482075,6.0,USDC,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x055B29979f6BC27669Ebd54182588FeF12ffBFc0,1.000000e+11
1367,1367,0x813Ac69E22c3D1c2A7CE95A7A641DB580b9363A0,0x95357303f995e184A7998dA6C6eA35cC728A1900,300000000000,17491767,6.0,USDC,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x9874f9B05B073771c7b98e192543FA2684e61246,6.000000e+10
1370,1370,0xF9eBC97A52E94DA558d4aEE46403D7ef7E465a46,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,120000000000000000000,17600969,18.0,WETH,0x67479449b2cf25AEE2fB6EF6f0aEc54591154F62,0x6e58309CD851A5B124E3A56768a42d12f3B6D104,2.000000e+19


In [12]:
data_cols = [x['name'] for x in df_abi[df_abi['name']=='getCreditAccountData']['abi'].values[0]['outputs'][0]['components']]

batchtime = datetime.utcnow()
df['batchtime'] = batchtime

for ids in list(chunks(list(df[pd.notna(df['Borrower'])].loc[:,'id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    try:
        d_data = get_data_multicall(w3_eth, df.loc[id_range], 'getCreditAccountData', df_abi, DataCompressor)
    except ContractLogicError:
        logging.error('getCreditAccountData: Multicall error, calling it one by one')
        d_data = {x:getCreditAccountData(x,y,z) for x,y,z in zip(df.loc[id_range]['id'],df.loc[id_range]['CM'],df.loc[id_range]['Borrower'])}
        d_data = AttributeDict.recursive(d_data)
    for data in data_cols:
        if data not in ['balances', 'inUse', 'borrower', 'addr', 'borrower', 'creditManager', 'since']: #duplicate columns
            df.loc[id_range, data] = df.loc[id_range].apply(lambda x: {y:z for y, z in zip(data_cols, d_data[x['id']])}[data] if d_data[x['id']] else None
                                                            , axis=1)
        
    tokens_to_frame = [allowedTokens[x]['symbol'] for x in allowedTokens] 
    for token in tokens_to_frame:
        df.loc[id_range, 'Balance_'+token] = df.loc[id_range].apply(lambda x: get_token_balance(x, token, data_cols, d_data, allowedTokens)
                                                        , axis=1)


In [13]:
display(df) 

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral,...,Balance_FXS,Balance_LQTY,Balance_stkcvxFRAX3CRV-f,Balance_stkcvxLUSD3CRV-f,Balance_stkcvxgusd3CRV,Balance_stkcvxcrvPlain3andSUSD,Balance_stkcvx3Crv,Balance_stkcvxcrvFRAX,Balance_stkcvxsteCRV,Balance_wstETH
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13864162,18.0,WETH,None,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
display(df.columns)

Index(['id', 'CA', 'CM', 'borrowedAmount', 'Since', 'Decimals', 'Symbol', 'CF',
       'Borrower', 'Collateral', 'batchtime', 'inUse', 'underlying',
       'borrowedAmountPlusInterest', 'borrowedAmountPlusInterestAndFees',
       'totalValue', 'healthFactor', 'borrowRate', 'repayAmount',
       'liquidationAmount', 'canBeClosed', 'cumulativeIndexAtOpen', 'version',
       'enabledTokenMask', 'Balance_DAI', 'Balance_1INCH', 'Balance_AAVE',
       'Balance_COMP', 'Balance_DPI', 'Balance_FEI', 'Balance_LINK',
       'Balance_SNX', 'Balance_UNI', 'Balance_USDC', 'Balance_WBTC',
       'Balance_WETH', 'Balance_YFI', 'Balance_yvDAI', 'Balance_yvUSDC',
       'Balance_CRV', 'Balance_SUSHI', 'Balance_LDO', 'Balance_FTM',
       'Balance_LUNA', 'Balance_G-OBS', 'Balance_G-BLOCK', 'Balance_stETH',
       'Balance_USDT', 'Balance_sUSD', 'Balance_FRAX', 'Balance_GUSD',
       'Balance_LUSD', 'Balance_steCRV', 'Balance_cvxsteCRV', 'Balance_3Crv',
       'Balance_cvx3Crv', 'Balance_FRAX3CRV-f', 'Bal

In [15]:
display(df[pd.notna(df['Borrower'])]) # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral,...,Balance_FXS,Balance_LQTY,Balance_stkcvxFRAX3CRV-f,Balance_stkcvxLUSD3CRV-f,Balance_stkcvxgusd3CRV,Balance_stkcvxcrvPlain3andSUSD,Balance_stkcvx3Crv,Balance_stkcvxcrvFRAX,Balance_stkcvxsteCRV,Balance_wstETH
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,None,0x896b94f4f27f12369698C302e2049cAe86936BbB,1.000000e+18,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,None,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,1.000000e+18,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
31,31,0xdAC87546b0ea6A448c7F3b69e079b0a616F4CC10,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,5000000000000000000,13868591,18.0,WETH,None,0x99BC02c239025E431D5741cC1DbA8CE77fc51CE3,2.500000e+18,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
33,33,0xB8c3FFFE2E13EFd96594EC6Ea9bBE0b2ABA6210e,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13869000,18.0,WETH,None,0x2bb8Ab3C2A9837de97A83c228A07E16928B4f07f,5.000000e+17,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
87,87,0xf74e4AEAA3B75c0662bF6024dcA385bF5fDBa8c0,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13884707,18.0,WETH,None,0x8CBC2D4c114D6f4411bBC991cE68Db994e45Eb4F,3.000000e+17,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,1363,0x8137351530f9CD81C87EAe9457A6CAF9f910f000,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,468920089983699958680,17321436,18.0,WETH,0x67479449b2cf25AEE2fB6EF6f0aEc54591154F62,0x316BE293C8f2380769e7b7e7382679FE5a3b6600,9.999015e+19,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0
1365,1365,0x2C0C914DC4ed0d850dDdb978ee04F4a3Ff445dc0,0x95357303f995e184A7998dA6C6eA35cC728A1900,300000000000,17482075,6.0,USDC,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x055B29979f6BC27669Ebd54182588FeF12ffBFc0,1.000000e+11,...,2308468535450713983290,0.0,0,0,0,0,0.0,0.0,0,0.0
1367,1367,0x813Ac69E22c3D1c2A7CE95A7A641DB580b9363A0,0x95357303f995e184A7998dA6C6eA35cC728A1900,300000000000,17491767,6.0,USDC,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x9874f9B05B073771c7b98e192543FA2684e61246,6.000000e+10,...,0,0.0,0,350699387406909024167277,0,0,0.0,0.0,0,0.0
1370,1370,0xF9eBC97A52E94DA558d4aEE46403D7ef7E465a46,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,120000000000000000000,17600969,18.0,WETH,0x67479449b2cf25AEE2fB6EF6f0aEc54591154F62,0x6e58309CD851A5B124E3A56768a42d12f3B6D104,2.000000e+19,...,0,0.0,0,0,0,0,0.0,0.0,0,0.0


In [16]:
logging.info(f'batchtime {batchtime}' )
logging.info(f'countCreditAccounts - countCreditAccountsInStock = {countCreditAccounts - countCreditAccountsInStock}')

2023-07-09 21:56:38 <module> INFO batchtime 2023-07-09 09:56:26.053598
2023-07-09 21:56:38 <module> INFO countCreditAccounts - countCreditAccountsInStock = 81


In [17]:
df_price_oracle = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_price_oracle = df_price_oracle.set_index('symbol', drop=True).transpose()
df_price_oracle.columns = ['Price_'+x for x in df_price_oracle.columns]
df_price_oracle[[x.replace('Price','Decimals') for x in df_price_oracle.columns]] = df_price_oracle.loc['decimals',:]
df_price_oracle = df_price_oracle.drop(index = 'decimals')

df_price_oracle = df_price_oracle.reset_index()
df_price_oracle = df_price_oracle.rename(columns={'index': 'Price_Asset'})
df_price_oracle['Price_Asset'] = df_price_oracle['Price_Asset'].apply(lambda x: x.replace('Price_',''))

df_price_oracle['batchtime'] = batchtime


In [18]:
display(df_price_oracle)

,Price_Asset,Price_DAI,Price_1INCH,Price_AAVE,Price_COMP,Price_DPI,Price_FEI,Price_LINK,Price_SNX,Price_UNI,...,Decimals_LQTY,Decimals_stkcvxFRAX3CRV-f,Decimals_stkcvxLUSD3CRV-f,Decimals_stkcvxgusd3CRV,Decimals_stkcvxcrvPlain3andSUSD,Decimals_stkcvx3Crv,Decimals_stkcvxcrvFRAX,Decimals_stkcvxsteCRV,Decimals_wstETH,batchtime
0,USD,0.999904,0.306882,72.68227,52.31638,70.79502,0.953969,6.176639,2.076,5.248458,...,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,2023-07-09 09:56:26.053598


In [19]:
df_token_price = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_token_price.columns = [x.replace('Price_','') for x in df_token_price.columns]
df_token_price['batchtime'] = batchtime
df_token_price = df_token_price.rename(columns = {'symbol':'token'})
numeric_cols = [x for x in df_token_price.columns if x not in ['token', 'batchtime','decimals']]
df_token_price[numeric_cols] = df_token_price[numeric_cols]
df_token_price[numeric_cols] = df_token_price[numeric_cols].astype('float64')

In [20]:
display(df_token_price)

,token,decimals,USD,batchtime
0,DAI,18,0.999904,2023-07-09 09:56:26.053598
1,1INCH,18,0.306882,2023-07-09 09:56:26.053598
2,AAVE,18,72.682270,2023-07-09 09:56:26.053598
3,COMP,18,52.316380,2023-07-09 09:56:26.053598
4,DPI,18,70.795020,2023-07-09 09:56:26.053598
5,FEI,18,0.953969,2023-07-09 09:56:26.053598
6,LINK,18,6.176639,2023-07-09 09:56:26.053598
7,SNX,18,2.076000,2023-07-09 09:56:26.053598
8,UNI,18,5.248458,2023-07-09 09:56:26.053598
9,USDC,6,1.000035,2023-07-09 09:56:26.053598


In [65]:
#Open, Close, Repay, Liquidate
OpenCreditAccount_topic             =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
OpenCreditAccount_topic_v1          =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic                 =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic_v1              =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic            =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='RepayCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic        =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic_v1     =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateExpiredCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateExpiredCreditAccount')  & (df_abi['contract_type']=='creditFacade') &(df_abi['type']=='event')]['topic'].values[0]

from_block = 0

logging.info(f'from_block={from_block}')

df_events = pd.DataFrame()

for CM in CreditManagers:
    logging.info(f'get_logs from CM {CM}')    
    logs_v1 = get_logs(w3_eth, CM, df_abi, 
                    [[OpenCreditAccount_topic_v1,
                      AddCollateral_topic_v1,
                      CloseCreditAccount_topic_v1,
                      RepayCreditAccount_topic_v1,
                      LiquidateCreditAccount_topic_v1,]
                    ],
                    from_block,
                    'latest')
    df_events = df_events.append(logs_v1, ignore_index = True)  
        
    if cm_dict[CM]['creditFacade']:
        for cf in cm_dict[CM]['creditFacade_upgraded']:
            logging.info(f'get_logs from CF {cf}') 
            logs = get_logs(w3_eth, cf, df_abi, 
                            [[OpenCreditAccount_topic,
                              AddCollateral_topic, 
                              CloseCreditAccount_topic, 
                              LiquidateCreditAccount_topic, 
                              LiquidateExpiredCreditAccount_topic]
                            ],
                            from_block,
                            'latest')
            df_events = df_events.append(logs, ignore_index = True)  
        
logging.info(f'number of events={len(df_events)}')
cf_dict = {x:cm_dict[x]['creditFacade_upgraded'] for x in cm_dict if 'creditFacade_upgraded' in cm_dict[x]}
cf_dict = {x:cf_dict[x] for x in cf_dict if len(cf_dict[x])>0}
cf_dict = {y:{'CreditManager':x} for x in cf_dict for y in cf_dict[x]}
logging.info(f'cf_dict = {cf_dict}')

if len(df_events)>0:
    df_events['CF'] = df_events['address'].apply(lambda x: x if x in cf_dict else None)
    df_events['CM'] = df_events['address'].apply(lambda x: x if x in cm_dict else cf_dict[x]['CreditManager'])
    
    
    df_events['CM_Token'] = df_events['CM'].apply(lambda x: cm_dict[x]['symbol'])
    df_events['blockHash'] = df_events['blockHash'].apply(lambda x: x.hex())
    df_events['transactionHash'] = df_events['transactionHash'].apply(lambda x: x.hex())
    df_events['Borrower'] = df_events.apply(lambda x: x['args']['to'] 
                                                  if x['event'] in ['CloseCreditAccount','CloseCreditAccount'] 
                                                  else x['args']['onBehalfOf'] if x['event'] in ['OpenCreditAccount','AddCollateral']
                                                  else x['args']['borrower'] if x['event'] in ['IncreaseBorrowedAmount']
                                                  else x['args']['owner'] if 'owner' in x['args'] and x['event'] in ['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']
                                                  else x['args']['borrower'] if 'borrower' in x['args'] and x['event'] in ['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Liquidator'] = df_events.apply(lambda x: x['args']['liquidator'] if x['event'] in ['LiquidateCreditAccount','LiquidateExpiredCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Collateral_Token'] = df_events.apply(lambda x: allowedTokens[x['args']['token']]['symbol'] if x['event']=='AddCollateral'
                                                    else cm_dict[x['CM']]['symbol']
                                        ,axis=1)
    df_events['Collateral'] = df_events.apply(lambda x: x['args']['amount']*(10**-cm_dict[x['CM']]['decimals']) if x['event']=='OpenCreditAccount' and 'amount' in x['args']
                                                    else x['args']['value']*(10**-allowedTokens[x['args']['token']]['decimals']) if x['event']=='AddCollateral'
                                                    else None
                                        ,axis=1)

    logging.info('pull event timestamp begin')
    df_events['blockTimestamp'] = df_events['blockNumber'].apply(lambda x: datetime.utcfromtimestamp(w3_eth.eth.getBlock(x)['timestamp']))
    logging.info('pull event timestamp end')
    
    logging.info('pull transaction fee begin')
    df_events['transaction_receipt'] = df_events['transactionHash'].apply(lambda x: dict(w3_eth.eth.get_transaction_receipt(x)))
    df_events['transaction_fee'] = df_events['transaction_receipt'].apply(lambda x: x['gasUsed']*x['effectiveGasPrice']/1e18)
    logging.info('pull transaction fee end')    
    
    #df_events[['args','transaction_receipt']] = df_events[['args','transaction_receipt']].apply(str)
    #df_events['Liquidator'] = df_events['Liquidator'].astype('object')    
    #df_events['Collateral'] = df_events['Collateral'].astype('float64')
    df_events.drop(columns = 'logIndex')
    df_events = df_events.sort_values('blockTimestamp', ignore_index = True)

else:
    logging.info(f'No new events since block {from_block}')
    

2023-07-09 22:30:03 <module> INFO from_block=0
2023-07-09 22:30:03 <module> INFO get_logs from CM 0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB
2023-07-09 22:30:04 <module> INFO get_logs from CM 0x2664cc24CBAd28749B3Dd6fC97A6B402484De527
2023-07-09 22:30:05 <module> INFO get_logs from CM 0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0
2023-07-09 22:30:07 <module> INFO get_logs from CM 0xC38478B0A4bAFE964C3526EEFF534d70E1E09017
2023-07-09 22:30:07 <module> INFO get_logs from CM 0x672461Bfc20DD783444a830Ad4c38b345aB6E2f7
2023-07-09 22:30:07 <module> INFO get_logs from CF 0xf6f4F24ae50206A07B8B32629AeB6cb1837d854F
2023-07-09 22:30:08 <module> INFO get_logs from CF 0xd4fe3eD38250C38A0094224C4B0224b5D5d0e7d9
2023-07-09 22:30:08 <module> INFO get_logs from CM 0x95357303f995e184A7998dA6C6eA35cC728A1900
2023-07-09 22:30:08 <module> INFO get_logs from CF 0x61fbb350e39cc7bF22C01A469cf03085774184aa
2023-07-09 22:30:09 <module> INFO get_logs from CF 0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245
2023-07-09 22

In [66]:
display(df_events)

,address,args,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex,CF,CM,CM_Token,Borrower,Liquidator,Collateral_Token,Collateral,blockTimestamp,transaction_receipt,transaction_fee
0,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"(sender, onBehalfOf, creditAccount, amount, bo...",0x6191b1dc0938794510eb96bc4e9b1ec8d1a91fa9352b...,13858003,OpenCreditAccount,316,0xc322fdd8d731a58b3e95414d3201fa53be82fd780e82...,220,None,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,None,DAI,1000.000000,2021-12-22 23:40:25,{'blockHash': b'a\x91\xb1\xdc\t8yE\x10\xeb\x96...,0.029386
1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(sender, onBehalfOf, creditAccount, amount, bo...",0x880436c3d9131fc9a17a6d79ad2acdcd50974d1cde97...,13862947,OpenCreditAccount,458,0x37aa66296c27993bdc5e81e27c6d819e7466df0c070c...,356,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,1.250000,2021-12-23 18:09:38,{'blockHash': b'\x88\x046\xc3\xd9\x13\x1f\xc9\...,0.047534
2,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"(sender, onBehalfOf, creditAccount, amount, bo...",0xef59810b1015fa4f1a64d68a6166fe9d23e40cb1f585...,13862953,OpenCreditAccount,168,0xae23bb735281dbfede5cc3352dad8b7bdaa529b27989...,125,None,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,DAI,0x7BAFC0D5c5892f2041FD9F2415A7611042218e22,None,DAI,1000.000000,2021-12-23 18:11:05,{'blockHash': b'\xefY\x81\x0b\x10\x15\xfaO\x1a...,0.036783
3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(onBehalfOf, token, value)",0x60a3c671e4f5d06281611a576410de04c4ec1478971b...,13862954,AddCollateral,434,0xf37a9984138c6b9c02bc657593ef6d0680dd69e2f002...,312,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,0.550000,2021-12-23 18:11:07,{'blockHash': b'`\xa3\xc6q\xe4\xf5\xd0b\x81a\x...,0.011669
4,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(owner, to, remainingFunds)",0x6db00f3e2855ea71eb99701dfd5adb413666e13b2e72...,13862971,CloseCreditAccount,300,0x380f4f2acff142b5a407b51cabbd50b611d9702aa99b...,160,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,NaN,2021-12-23 18:14:12,{'blockHash': b'm\xb0\x0f>(U\xeaq\xeb\x99p\x1d...,0.029957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105,0xC59135f449bb623501145443c70A30eE648Fa304,"(onBehalfOf, creditAccount, borrowAmount, refe...",0xc97d2ee1e1f1babc0702a28bbfd0183e840ccc2861fb...,17600969,OpenCreditAccount,173,0x94673eb6fd9f4b06aec2203fbcb5ed05f9b47b922c10...,103,0xC59135f449bb623501145443c70A30eE648Fa304,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,WETH,0x6e58309CD851A5B124E3A56768a42d12f3B6D104,None,WETH,NaN,2023-07-01 18:26:11,{'blockHash': b'\xc9}.\xe1\xe1\xf1\xba\xbc\x07...,0.008769
4106,0xC59135f449bb623501145443c70A30eE648Fa304,"(onBehalfOf, token, value)",0xc97d2ee1e1f1babc0702a28bbfd0183e840ccc2861fb...,17600969,AddCollateral,176,0x94673eb6fd9f4b06aec2203fbcb5ed05f9b47b922c10...,103,0xC59135f449bb623501145443c70A30eE648Fa304,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,WETH,0x6e58309CD851A5B124E3A56768a42d12f3B6D104,None,WETH,20.000000,2023-07-01 18:26:11,{'blockHash': b'\xc9}.\xe1\xe1\xf1\xba\xbc\x07...,0.008769
4107,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,"(onBehalfOf, creditAccount, borrowAmount, refe...",0xa456ce921940a097d2d78c8a9cf2985f351c0cb58145...,17647010,OpenCreditAccount,149,0x20d0e8943c2455d3495954fd12321fd24f5e14a0cf16...,53,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x95357303f995e184A7998dA6C6eA35cC728A1900,USDC,0x166E58eA56399d5809FFfe195196554E9934bfF7,None,USDC,NaN,2023-07-08 05:36:23,{'blockHash': b'\xa4V\xce\x92\x19@\xa0\x97\xd2...,0.005985
4108,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,"(onBehalfOf, token, value)",0xa456ce921940a097d2d78c8a9cf2985f351c0cb58145...,17647010,AddCollateral,152,0x20d0e8943c2455d3495954fd12321fd24f5e14a0cf16...,53,0x8d2f33d168cca6D2436De16c27d3f1cEa30aC245,0x95357303f995e184A7998dA6C6eA35c

# Data Studio Report
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0